<a href="https://colab.research.google.com/github/ZainebEzzahi/DAIN_/blob/main/DAIN_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. A bunch of setup... (builds take a while)
## References:
> Repo: https://github.com/baowenbo/DAIN

> Project: https://sites.google.com/view/wenbobao/dain

In [ ]:
!git clone https://github.com/baowenbo/DAIN.git
!pip install scipy==1.1.0
!pip install --force tqdm==4.24.0

Cloning into 'DAIN'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 500 (delta 4), reused 1 (delta 0), pack-reused 491
Receiving objects: 100% (500/500), 171.43 KiB | 3.36 MiB/s, done.
Resolving deltas: 100% (267/267), done.
     |████████████████████████████████| 31.2MB 113kB/s 
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
     |████████████████████████████████| 51kB 2.1MB/s 
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


### Check what GPU is under the hood:

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1b4c1df5-1f4b-675c-c4b6-8fbfbe3202f5)


## I'm guessing we need to adjust the default nvcc_args...

In [ ]:
%%writefile /content/DAIN/my_package/DepthFlowProjection/setup.py

#!/usr/bin/env python3
import os
import torch

from setuptools import setup, find_packages
from torch.utils.cpp_extension import BuildExtension, CUDAExtension

cxx_args = ['-std=c++11']
nvcc_args = [
    '-gencode', 'arch=compute_50,code=sm_50',
    '-gencode', 'arch=compute_52,code=sm_52',
    '-gencode', 'arch=compute_60,code=sm_60',
    '-gencode', 'arch=compute_61,code=sm_61'
    # '-gencode', 'arch=compute_70,code=sm_70',
    '-gencode', 'arch=compute_70,code=compute_70'
]

setup(
    name='depthflowprojection_cuda',
    ext_modules=[
        CUDAExtension('depthflowprojection_cuda', [
            'depthflowprojection_cuda.cc',
            'depthflowprojection_cuda_kernel.cu'
        ], extra_compile_args={'cxx': cxx_args, 'nvcc': nvcc_args})
    ],
    cmdclass={
        'build_ext': BuildExtension
    })

Overwriting /content/DAIN/my_package/DepthFlowProjection/setup.py


# Downgrade gcc:

In [ ]:
!gcc -dumpversion
!g++ -dumpversion

### Repo says to use gcc 4.9.1 but wasn't able to install that one...

In [ ]:
!sudo apt-get install gcc-6 g++-6

## Remove & overwrite any existing symbolic links to activate freshly installed gcc:

In [ ]:
!rm /usr/bin/gcc
!rm /usr/bin/g++

!ln -s /usr/bin/gcc-6 /usr/bin/gcc
!ln -s /usr/bin/g++-6 /usr/bin/g++

!gcc -dumpversion

## Downgrade to Cuda 9.0: (Colab uses 10.0 which doesn't work with DAIN)

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1
!echo ****** Cuda reinstall completed. Restarting runtime now! *******
import os
os.kill(os.getpid(), 9)

--2020-02-07 15:37:03--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?L0a274Dm3KH6Q2JELxhR1XN6Jpdg5Mt9jlYP8s8mIhOuvj8-bMYTqh39Tr1AThV2gV1KKvHBzKC0HuvxiHssm9ii1KwhaN9WPA-7wJoDUB5gFFyZbIzfEoBVJMdZRtSHOwhUlahEGsVrAtI3JPKi5xfgfkcjTB0yMofS_YYM0E7Ly26TlxR5ouIzAIFg4_UCThoThNwbjG3kLMJvqz8j [following]
--2020-02-07 15:37:03--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?L0a274Dm3KH6Q2JELxhR1XN6Jpdg5Mt9jlYP8s8mIhOuvj8-bMYTqh39Tr1AThV2gV1KKvHBzKC0HuvxiHssm9ii1KwhaN9WPA-7wJoDUB5gFF

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:03_CDT_2017
Cuda compilation tools, release 9.0, V9.0.176


# Downgrade Torch version:

In [ ]:
!yes | pip uninstall torch torchvision

Uninstalling torch-1.0.0:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/lib/python3.6/dist-packages/caffe2/*
    /usr/local/lib/python3.6/dist-packages/torch-1.0.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/torch/*
Proceed (y/n)?   Successfully uninstalled torch-1.0.0
Uninstalling torchvision-0.2.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/torchvision-0.2.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/torchvision/*
Proceed (y/n)?   Successfully uninstalled torchvision-0.2.1


In [ ]:
!pip install torch==1.0.0 torchvision==0.2.1

import torch
print(torch.__version__)

     |████████████████████████████████| 591.8MB 31kB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
1.0.0


# Build the package: (this is where the errors start...)

In [ ]:
%cd '/content/DAIN/my_package'
!./build.sh

/content/DAIN/my_package
Need pytorch>=1.0.0
./build.sh: line 4: activate: No such file or directory
running install
running bdist_egg
running egg_info
creating mindepthflowprojection_cuda.egg-info
writing mindepthflowprojection_cuda.egg-info/PKG-INFO
writing dependency_links to mindepthflowprojection_cuda.egg-info/dependency_links.txt
writing top-level names to mindepthflowprojection_cuda.egg-info/top_level.txt
writing manifest file 'mindepthflowprojection_cuda.egg-info/SOURCES.txt'
writing manifest file 'mindepthflowprojection_cuda.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
building 'mindepthflowprojection_cuda' extension
creating build
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/torch/lib/include -I/usr/local/lib/python3.

In [ ]:
%cd '/content/DAIN/PWCNet/correlation_package_pytorch1_0'
!./build.sh

/content/DAIN/PWCNet/correlation_package_pytorch1_0
Need pytorch>=1.0.0
./build.sh: line 4: activate: No such file or directory
running install
running bdist_egg
running egg_info
creating correlation_cuda.egg-info
writing correlation_cuda.egg-info/PKG-INFO
writing dependency_links to correlation_cuda.egg-info/dependency_links.txt
writing top-level names to correlation_cuda.egg-info/top_level.txt
writing manifest file 'correlation_cuda.egg-info/SOURCES.txt'
writing manifest file 'correlation_cuda.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
building 'correlation_cuda' extension
creating build
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/torch/include -I/usr/local/lib/python3.6/dist-packages/torch/include/torch/csrc/api/include -

## Download pretrained model:

In [ ]:
%cd '/content/DAIN'
!mkdir model_weights
!mkdir MiddleBurySet

/content/DAIN


In [ ]:
%cd '/content/DAIN/model_weights'
!wget http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

/content/DAIN/model_weights
--2020-02-07 15:18:13--  http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth
Resolving vllab1.ucmerced.edu (vllab1.ucmerced.edu)... 169.236.184.68
Connecting to vllab1.ucmerced.edu (vllab1.ucmerced.edu)|169.236.184.68|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96319643 (92M)
Saving to: ‘best.pth’

best.pth            100%[===================>]  91.86M  18.3MB/s    in 5.9s    

2020-02-07 15:18:19 (15.4 MB/s) - ‘best.pth’ saved [96319643/96319643]



In [ ]:
%cd '/content/DAIN/MiddleBurySet'
!wget http://vision.middlebury.edu/flow/data/comp/zip/other-color-allframes.zip
!unzip other-color-allframes.zip
!wget http://vision.middlebury.edu/flow/data/comp/zip/other-gt-interp.zip
!unzip other-gt-interp.zip

/content/DAIN/MiddleBurySet
--2020-02-07 15:18:21--  http://vision.middlebury.edu/flow/data/comp/zip/other-color-allframes.zip
Resolving vision.middlebury.edu (vision.middlebury.edu)... 140.233.20.14
Connecting to vision.middlebury.edu (vision.middlebury.edu)|140.233.20.14|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33671986 (32M) [application/zip]
Saving to: ‘other-color-allframes.zip’

other-color-allfram 100%[===================>]  32.11M  20.2MB/s    in 1.6s    

2020-02-07 15:18:23 (20.2 MB/s) - ‘other-color-allframes.zip’ saved [33671986/33671986]

Archive:  other-color-allframes.zip
  inflating: other-data/Beanbags/frame07.png  
  inflating: other-data/Beanbags/frame08.png  
  inflating: other-data/Beanbags/frame09.png  
  inflating: other-data/Beanbags/frame10.png  
  inflating: other-data/Beanbags/frame11.png  
  inflating: other-data/Beanbags/frame12.png  
  inflating: other-data/Beanbags/frame13.png  
  inflating: other-data/Beanbags/frame14.png 

In [ ]:
%cd '/content/DAIN'
!CUDA_VISIBLE_DEVICES=0 python demo_MiddleBury.py

/content/DAIN
Traceback (most recent call last):
  File "demo_MiddleBury.py", line 10, in <module>
    import networks
  File "/content/DAIN/networks/__init__.py", line 1, in <module>
    from .DAIN import DAIN
  File "/content/DAIN/networks/DAIN.py", line 4, in <module>
    from my_package.FilterInterpolation import  FilterInterpolationModule
  File "/content/DAIN/my_package/FilterInterpolation/__init__.py", line 1, in <module>
    from .FilterInterpolationModule import *
  File "/content/DAIN/my_package/FilterInterpolation/FilterInterpolationModule.py", line 6, in <module>
    from .FilterInterpolationLayer import FilterInterpolationLayer,WeightLayer, PixelValueLayer,PixelWeightLayer,ReliableWeightLayer
  File "/content/DAIN/my_package/FilterInterpolation/FilterInterpolationLayer.py", line 4, in <module>
    import filterinterpolation_cuda as my_lib
ModuleNotFoundError: No module named 'filterinterpolation_cuda'
